In [1]:
# Import Python Tools:
import pandas as pd
import numpy as np
from numpy import sqrt, pi, average, cos, sin, tan, arcsin, arccos, arctan, deg2rad, rad2deg
# import sectionproperties as shape
from WhatIfAnalysis import GoalSeek

from ASCE import load_combos

from steel_design.AISC import aisc_W, aisc_HSS
steel_density = 0.2833 #pci

from concrete_design import material_properties as conc
from concrete_design.steel_reinforcement import rebar_df as rebar
conc_density = 150 #pcf

from masonry_design import material_properties as cmu
from masonry_design.material_properties import NW_CMU #psf

<div style="page-break-after: always;"></div>

# Brace Connection Design (UFM)

$α - β \tan(θ) = e_b \tan(θ) - e_c$

In [2]:
roof_beam = "W18X35"
floor_beam = "W27X94"
column = "W14X61"

In [3]:
bay_width = 25 #ft
bay_height = 15.5 #ft
θ = arctan(bay_width/bay_height)
f"θ = {round(rad2deg(θ), 2)} deg"

'θ = 58.2 deg'

In [4]:
P_r = 36 #kips (required brace axial force)
H_r = P_r * sin(θ) #kips
V_r = P_r * cos(θ) #kips

f"H_r = {round(H_r, 2)} kips, V_r = {round(V_r, 2)} kips"

'H_r = 30.6 kips, V_r = 18.97 kips'

## Brace Connection at Floor Beam

### Force Determination for Gusset

In [5]:
d_flrbm = aisc_W.loc[floor_beam, "d"] #in
e_b = d_flrbm/2 #in

d_col = aisc_W.loc[column, "d"] #in
e_c = d_col/2 #in

f"floor beam d = {d_flrbm} in, column d = {d_col} in, e_b = {e_b} in, e_c = {e_c} in"

'floor beam d = 26.9 in, column d = 13.9 in, e_b = 13.45 in, e_c = 6.95 in'

In [6]:
β = 7 #in

In [7]:
def function(α):
    return α - β * tan(θ)
α = GoalSeek(function, e_b * tan(θ) - e_c, x0=27)
α

26.033854166666664

In [8]:
α - β * tan(θ), e_b * tan(θ) - e_c

(14.743531586021508, 14.743548387096766)

In [9]:
r = sqrt((α + e_c)**2 + (β + e_b)**2)

V_rc = β / r * P_r #kips
H_rc = e_c / r * P_r #kips
V_rb = e_b / r * P_r #kips
H_rb = α / r * P_r #kips

In [10]:
print(f"""V_rc = {round(V_rc, 2)} kips, H_rc = {round(H_rc, 2)} kips, 
V_rb = {round(V_rb, 2)} kips, H_rb = {round(H_rb, 2)} kips""")

V_rc = 6.49 kips, H_rc = 6.45 kips, 
V_rb = 12.48 kips, H_rb = 24.15 kips


### Gusset Weld Design

In [11]:
φ = 0.75
F_nBM = 50 #ksi
F_EXX = 70 #ksi
F_nw = 0.60 * F_EXX #ksi

#### Beam Side

In [12]:
t_f = aisc_W.loc[floor_beam, "tf"]
l_w = α * 2 #in
A_BM = t_f * l_w #in^2

s = 1/4 #in
A_we = l_w * s * sqrt(2)/2 #in^2
print(f"t_f = {t_f} in, l_w = {round(l_w, 2)} in")
print(f"A_BM = {round(A_BM, 2)} in^2, A_we = {round(A_we, 2)} in^2")

t_f = 0.745 in, l_w = 52.07 in
A_BM = 38.79 in^2, A_we = 9.2 in^2


In [13]:
θ_weld = arctan(V_rb / H_rb) #rad
k_ds = (1.0 + 0.50 * sin(θ_weld)**1.5)
print(f"θ_weld = {round(rad2deg(θ_weld), 2)} deg, k_ds = {round(k_ds, 2)}")

φR_n_BM = φ * F_nBM * A_BM #kips
φR_n_weld = φ * F_nw * A_we * k_ds #kips

print(f"φR_n_BM = {round(φR_n_BM, 2)} kips, φR_n_weld = {round(φR_n_weld, 2)} kips")

θ_weld = 27.32 deg, k_ds = 1.16
φR_n_BM = 1454.64 kips, φR_n_weld = 335.02 kips


In [14]:
R_u = sqrt(V_rb**2 + H_rb**2) #kips
f"R_u = {round(R_u, 2)} kips"

'R_u = 27.18 kips'

In [15]:
R_u / φR_n_BM, R_u / φR_n_weld

(0.018686410365933284, 0.08113605481064265)

#### Column Side

In [16]:
t_f = aisc_W.loc[column, "tf"]
l_w = β * 2 #in
A_BM = t_f * l_w #in^2

s = 1/4 #in
A_we = l_w * s * sqrt(2)/2 #in^2
print(f"t_f = {t_f} in, l_w = {round(l_w, 2)} in")
print(f"A_BM = {round(A_BM, 2)} in^2, A_we = {round(A_we, 2)} in^2")

t_f = 0.645 in, l_w = 14 in
A_BM = 9.03 in^2, A_we = 2.47 in^2


In [17]:
θ_weld = arctan(V_rc / H_rc) #rad
k_ds = (1.0 + 0.50 * sin(θ_weld)**1.5)
print(f"θ_weld = {round(rad2deg(θ_weld), 2)} deg, k_ds = {round(k_ds, 2)}")

φR_n_BM = φ * F_nBM * A_BM #kips
φR_n_weld = φ * F_nw * A_we * k_ds #kips

print(f"φR_n_BM = {round(φR_n_BM, 2)} kips, φR_n_weld = {round(φR_n_weld, 2)} kips")

θ_weld = 45.21 deg, k_ds = 1.3
φR_n_BM = 338.63 kips, φR_n_weld = 101.26 kips


In [18]:
R_u = sqrt(V_rc**2 + H_rc**2) #kips
f"R_u = {round(R_u, 2)} kips"

'R_u = 9.15 kips'

In [19]:
R_u / φR_n_BM, R_u / φR_n_weld

(0.027021734575210954, 0.0903635637688743)

### Beam and Column Design

In [20]:
R_rb = 22.22 #kips
R_rc = 20 #kips
A_rb = 75 #kips

In [21]:
Vr_bm_to_col = R_rb - V_rb #kips
Ar_bm_to_col = A_rb + (H_r - H_rb) #kips

print(f"required V bm-to-col conn = {round(Vr_bm_to_col, 2)} kips")
print(f"required A bm-to-col conn = {round(Ar_bm_to_col, 2)} kips")

required V bm-to-col conn = 9.74 kips
required A bm-to-col conn = 81.45 kips


--> RISA Connection Check

## Brace Connection at Roof Beam

In [22]:
d_rfbm = aisc_W.loc[roof_beam, "d"] #in
e_b = d_rfbm/2 #in

d_col = aisc_W.loc[column, "d"] #in
e_c = d_col/2 #in

f"floor beam d = {d_rfbm} in, column d = {d_col} in, e_b = {e_b} in, e_c = {e_c} in"

'floor beam d = 17.7 in, column d = 13.9 in, e_b = 8.85 in, e_c = 6.95 in'

In [23]:
β = 7 #in

In [24]:
def function(α):
    return α - β * tan(θ)
α = GoalSeek(function, e_b * tan(θ) - e_c, x0=27)
α

18.614583333333332

In [25]:
α*2

37.229166666666664

In [26]:
α - β * tan(θ), e_b * tan(θ) - e_c

(7.324260752688176, 7.324193548387091)

In [27]:
r = sqrt((α + e_c)**2 + (β + e_b)**2)

V_rc = β / r * P_r #kips
H_rc = e_c / r * P_r #kips
V_rb = e_b / r * P_r #kips
H_rb = α / r * P_r #kips

In [28]:
print(f"""V_rc = {round(V_rc, 2)} kips, H_rc = {round(H_rc, 2)} kips, 
V_rb = {round(V_rb, 2)} kips, H_rb = {round(H_rb, 2)} kips""")

V_rc = 8.38 kips, H_rc = 8.32 kips, 
V_rb = 10.59 kips, H_rb = 22.28 kips


### Gusset Weld Design

In [29]:
φ = 0.75
F_nBM = 50 #ksi
F_EXX = 70 #ksi
F_nw = 0.60 * F_EXX #ksi

#### Beam Side

In [30]:
t_f = aisc_W.loc[roof_beam, "tf"]
l_w = α * 2 #in
A_BM = t_f * l_w #in^2

s = 1/4 #in
A_we = l_w * s * sqrt(2)/2 #in^2
print(f"t_f = {t_f} in, l_w = {round(l_w, 2)} in")
print(f"A_BM = {round(A_BM, 2)} in^2, A_we = {round(A_we, 2)} in^2")

t_f = 0.425 in, l_w = 37.23 in
A_BM = 15.82 in^2, A_we = 6.58 in^2


In [31]:
θ_weld = arctan(V_rb / H_rb) #rad
k_ds = (1.0 + 0.50 * sin(θ_weld)**1.5)
print(f"θ_weld = {round(rad2deg(θ_weld), 2)} deg, k_ds = {round(k_ds, 2)}")

φR_n_BM = φ * F_nBM * A_BM #kips
φR_n_weld = φ * F_nw * A_we * k_ds #kips

print(f"φR_n_BM = {round(φR_n_BM, 2)} kips, φR_n_weld = {round(φR_n_weld, 2)} kips")

θ_weld = 25.43 deg, k_ds = 1.14
φR_n_BM = 593.34 kips, φR_n_weld = 236.47 kips


In [32]:
R_u = sqrt(V_rb**2 + H_rb**2) #kips
f"R_u = {round(R_u, 2)} kips"

'R_u = 24.67 kips'

In [33]:
R_u / φR_n_BM, R_u / φR_n_weld

(0.04157525604749542, 0.1043173090070362)

#### Column Side

In [34]:
t_f = aisc_W.loc[column, "tf"]
l_w = β * 2 #in
A_BM = t_f * l_w #in^2

s = 1/4 #in
A_we = l_w * s * sqrt(2)/2 #in^2
print(f"t_f = {t_f} in, l_w = {round(l_w, 2)} in")
print(f"A_BM = {round(A_BM, 2)} in^2, A_we = {round(A_we, 2)} in^2")

t_f = 0.645 in, l_w = 14 in
A_BM = 9.03 in^2, A_we = 2.47 in^2


In [35]:
θ_weld = arctan(V_rc / H_rc) #rad
k_ds = (1.0 + 0.50 * sin(θ_weld)**1.5)
print(f"θ_weld = {round(rad2deg(θ_weld), 2)} deg, k_ds = {round(k_ds, 2)}")

φR_n_BM = φ * F_nBM * A_BM #kips
φR_n_weld = φ * F_nw * A_we * k_ds #kips

print(f"φR_n_BM = {round(φR_n_BM, 2)} kips, φR_n_weld = {round(φR_n_weld, 2)} kips")

θ_weld = 45.21 deg, k_ds = 1.3
φR_n_BM = 338.63 kips, φR_n_weld = 101.26 kips


In [36]:
R_u = sqrt(V_rc**2 + H_rc**2) #kips
f"R_u = {round(R_u, 2)} kips"

'R_u = 11.81 kips'

In [37]:
R_u / φR_n_BM, R_u / φR_n_weld

(0.03486392551630428, 0.1165886870754762)

### Beam and Column Design

In [38]:
R_rb = 20 #kips
R_rc = 0 #kips
A_rb = 15 #kips

In [39]:
Vr_bm_to_col = R_rb - V_rb #kips
Ar_bm_to_col = A_rb + (H_r - H_rb) #kips

print(f"required V bm-to-col conn = {round(Vr_bm_to_col, 2)} kips")
print(f"required A bm-to-col conn = {round(Ar_bm_to_col, 2)} kips")

required V bm-to-col conn = 9.41 kips
required A bm-to-col conn = 23.32 kips


--> RISA Connection Check